In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import defaultdict
import string
import tensorflow as tf
import re
import os 
import time
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

2024-02-23 12:22:20.543739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-23 12:22:21.075965: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ENCODER_LEN = 500
DECODER_LEN = 100
BATCH_SIZE = 64
BUFFER_SIZE = BATCH_SIZE*8

# Load dataset

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("xsum", split="train")

/home/bobby/code-repo/practices/transformer-examples/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bobby/code-repo/practices/transformer-examples/env/lib/python3.8/site-packages/datasets/load.py:1454: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
raw_datasets = raw_datasets.train_test_split(
    train_size=0.1, test_size=0.1
)

In [5]:
train_df = pd.DataFrame(raw_datasets['train'])
train_df['train test'] = 'train'

df = pd.concat([train_df], axis = 0).reset_index(drop = True)

df

,document,summary,id,train test
0,"Sir Iain Lobban will leave later this year, af...",The head of GCHQ - Britain's electronic intell...,25937478,train
1,The bank's quarterly sectoral forecast said im...,The Northern Ireland economy should grow at 2....,27943149,train
2,Developed countries agreed to stop the subsidi...,Countries in the World Trade Organization (WTO...,35145377,train
3,They were trying to explain why the animals ha...,Elephants have enhanced defences against cance...,34466220,train
4,The new one will have 12 sides instead of a sm...,Pocket money may look a bit different this tim...,35935007,train
...,...,...,...,...
20399,It has advised all international postal servic...,The Republic of Ireland's postal service An Po...,34426129,train
20400,"It's a kind of all-purpose, good-for-all-occas...","The phrase ""out of touch political elite"" trip...",36562167,train
20401,Rachel and Nyomi Fee deny murdering two-year-o...,Evidence of untreated injuries found on Liam F...,36370492,train
20402,"The move, expected since December, also sees B...",Valtteri Bottas has succeeded retired world ch...,38489054,train


In [6]:
article = df['document']
summary = df['summary']
article = article.apply(lambda x: '<SOS> ' + x + ' <EOS>')
summary = summary.apply(lambda x: '<SOS> ' + x + ' <EOS>')

In [7]:
def preprocess(text):
    text = re.sub(r"&.[1-9]+;"," ",text)
    return text
article = article.apply(lambda x: preprocess(x))
summary = summary.apply(lambda x: preprocess(x))

In [8]:
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = '<unk>'
article_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token)
article_tokenizer.fit_on_texts(article)
summary_tokenizer.fit_on_texts(summary)
inputs = article_tokenizer.texts_to_sequences(article)
targets = summary_tokenizer.texts_to_sequences(summary)

In [9]:
ENCODER_VOCAB = len(article_tokenizer.word_index) + 1
DECODER_VOCAB = len(summary_tokenizer.word_index) + 1
print(ENCODER_VOCAB, DECODER_VOCAB)

114832 27383


In [10]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=ENCODER_LEN, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=DECODER_LEN, padding='post', truncating='post')
inputs = tf.cast(inputs, dtype=tf.int64)
targets = tf.cast(targets, dtype=tf.int64)

2024-02-23 12:23:34.763485: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 12:23:34.763748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 12:23:34.763961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


2024-02-23 12:23:40.671777: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 81616000 exceeds 10% of free system memory.


In [13]:
len(inputs[0].numpy())

500

In [12]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [14]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

In [15]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [16]:
## Attention 

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights
    
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

In [17]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

In [18]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

In [19]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x
    
class Decoder(tf.keras.layers.Layer):
        
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights

In [20]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights

In [42]:
num_layers = 8 # number of layers for the encoder and decoder
d_model = 256 # dimension of the model
dff = 1024 # dimension of the feed-forward network
num_heads = 8 # number of heads for the multihead attention 
dropout_rate = 0.1 # for regularization
EPOCHS = 20 # how many times to train over the input

In [43]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [44]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [45]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
    accuracies = tf.equal(real, tf.argmax(pred, axis=2))
    #accuracies = tf.cast(accuracies, dtype= tf.float32)

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [46]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [47]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=ENCODER_VOCAB,
    target_vocab_size=DECODER_VOCAB,
    pe_input=1000,
    pe_target=1000,
    rate=dropout_rate)

In [48]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask

In [49]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [50]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy_function(tar_real, predictions))

In [51]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        if batch % 100 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
   
    print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 10.2151 Accuracy 0.0000
Epoch 1 Batch 100 Loss 10.0225 Accuracy 0.0334
Epoch 1 Batch 200 Loss 9.7061 Accuracy 0.0389
Epoch 1 Batch 300 Loss 9.2681 Accuracy 0.0407
Epoch 1 Loss 9.1856 Accuracy 0.0409
Time taken for 1 epoch: 215.3429934978485 secs

Epoch 2 Batch 0 Loss 7.6548 Accuracy 0.0410
Epoch 2 Batch 100 Loss 7.3920 Accuracy 0.0443
Epoch 2 Batch 200 Loss 7.2947 Accuracy 0.0513
Epoch 2 Batch 300 Loss 7.2315 Accuracy 0.0566
Epoch 2 Loss 7.2205 Accuracy 0.0576
Time taken for 1 epoch: 172.02420663833618 secs

Epoch 3 Batch 0 Loss 7.1734 Accuracy 0.0577
Epoch 3 Batch 100 Loss 6.8963 Accuracy 0.0648
Epoch 3 Batch 200 Loss 6.7879 Accuracy 0.0718
Epoch 3 Batch 300 Loss 6.6716 Accuracy 0.0787
Epoch 3 Loss 6.6521 Accuracy 0.0799
Time taken for 1 epoch: 170.8056001663208 secs

Epoch 4 Batch 0 Loss 6.4380 Accuracy 0.0800
Epoch 4 Batch 100 Loss 6.2153 Accuracy 0.0867
Epoch 4 Batch 200 Loss 6.1610 Accuracy 0.0927
Epoch 4 Batch 300 Loss 6.1006 Accuracy 0.0985
Epoch 4 Loss 6.08

2024-02-23 13:24:40.249212: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117587968 exceeds 10% of free system memory.
2024-02-23 13:24:40.281549: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117587968 exceeds 10% of free system memory.
2024-02-23 13:24:40.573843: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117587968 exceeds 10% of free system memory.


Saving checkpoint for epoch 5 at checkpoints/ckpt-1
Epoch 5 Loss 5.7600 Accuracy 0.1152
Time taken for 1 epoch: 171.7627944946289 secs

Epoch 6 Batch 0 Loss 5.6226 Accuracy 0.1152
Epoch 6 Batch 100 Loss 5.5443 Accuracy 0.1196
Epoch 6 Batch 200 Loss 5.5128 Accuracy 0.1237
Epoch 6 Batch 300 Loss 5.4666 Accuracy 0.1277
Epoch 6 Loss 5.4599 Accuracy 0.1283
Time taken for 1 epoch: 169.78914713859558 secs

Epoch 7 Batch 0 Loss 5.3799 Accuracy 0.1283
Epoch 7 Batch 100 Loss 5.2735 Accuracy 0.1322
Epoch 7 Batch 200 Loss 5.2503 Accuracy 0.1356
Epoch 7 Batch 300 Loss 5.2103 Accuracy 0.1390
Epoch 7 Loss 5.2039 Accuracy 0.1396
Time taken for 1 epoch: 169.69656014442444 secs

Epoch 8 Batch 0 Loss 5.0017 Accuracy 0.1397
Epoch 8 Batch 100 Loss 5.0361 Accuracy 0.1430
Epoch 8 Batch 200 Loss 5.0227 Accuracy 0.1461
Epoch 8 Batch 300 Loss 4.9876 Accuracy 0.1491
Epoch 8 Loss 4.9814 Accuracy 0.1496
Time taken for 1 epoch: 169.61472511291504 secs

Epoch 9 Batch 0 Loss 4.7164 Accuracy 0.1497
Epoch 9 Batch 100 L

In [52]:
def evaluate(input_article):
    input_article = article_tokenizer.texts_to_sequences([input_article])
    input_article = tf.keras.preprocessing.sequence.pad_sequences(input_article, maxlen=ENCODER_LEN, 
                                                                   padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_article[0], 0)

    decoder_input = [summary_tokenizer.word_index['<sos>']]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(DECODER_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index['<eos>']:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [53]:
def summarize(input_article):
    summarized = evaluate(input_article=input_article)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  
    return summary_tokenizer.sequences_to_texts(summarized)[0]

In [40]:
print(article[0])

<SOS> Sir Iain Lobban will leave later this year, after six years as director.
The Foreign Office said Sir Iain, 53, was doing "outstanding job" and his departure was "planned".
Officials denied the move was linked to controversy over GCHQ and its US counterpart, the NSA, sparked by disclosures from former US intelligence contractor Edward Snowden.
Sir Iain, who first joined GCHQ in 1983, became director in June 2008.
"Today is simply about starting the process of ensuring we have a suitable successor in place before he moves on as planned at the end of the year," a Foreign Office spokesman said.
ln November, Sir Iain became the first head of the agency to give evidence in public when he appeared before MPs on the Intelligence and Security Committee, alongside the heads of MI5 and MI6.
They came under pressure to be more open after leaks by Mr Snowden revealed widespread spying by GCHQ and the US National Security Agency.
Sir Iain told the committee Mr Snowden's disclosures had done im

In [41]:
print("Real Headline : ", summary[0],"\n Predicted Summary : ", summarize(article[0]))


Real Headline :  <SOS> The head of GCHQ - Britain's electronic intelligence gathering agency - is to step down, the Foreign Office has said. <EOS> 
 Predicted Summary :  the uk is to be the first of the first time in the uk to pay the uk to pay jobs in the uk


In [54]:
print(article[0])

<SOS> Sir Iain Lobban will leave later this year, after six years as director.
The Foreign Office said Sir Iain, 53, was doing "outstanding job" and his departure was "planned".
Officials denied the move was linked to controversy over GCHQ and its US counterpart, the NSA, sparked by disclosures from former US intelligence contractor Edward Snowden.
Sir Iain, who first joined GCHQ in 1983, became director in June 2008.
"Today is simply about starting the process of ensuring we have a suitable successor in place before he moves on as planned at the end of the year," a Foreign Office spokesman said.
ln November, Sir Iain became the first head of the agency to give evidence in public when he appeared before MPs on the Intelligence and Security Committee, alongside the heads of MI5 and MI6.
They came under pressure to be more open after leaks by Mr Snowden revealed widespread spying by GCHQ and the US National Security Agency.
Sir Iain told the committee Mr Snowden's disclosures had done im

In [55]:
print("Real Headline : ", summary[0],"\n Predicted Summary : ", summarize(article[0]))


Real Headline :  <SOS> The head of GCHQ - Britain's electronic intelligence gathering agency - is to step down, the Foreign Office has said. <EOS> 
 Predicted Summary :  the us has launched a lawsuit on whether the us government to allow its ban to allow gay internet agencies from refugees


In [56]:
print(article[1])

<SOS> The bank's quarterly sectoral forecast said improvement in the jobs market and a number of foreign direct investments injected momentum into the recovery.
The data was recorded during the second quarter of this year.
However, the bank warned that continued austerity could force government departments to reduce staff numbers.
Northern Ireland's latest official jobs figures suggest that in the first quarter of this year the private sector added 3,860 jobs while the public sector lost 130 jobs.
Danske Bank's report pointed to strong growth in professional and business services like accountancy and recruitment.
Its chief economist Angela McGowan said there had been a "feel-good factor" in the economy in the second quarter of this year.
She said this could be seen in rising levels of household spending and improved levels of business investment.
However, she added that households may feel "a little cheated" as the economic improvements are still not translating into rising earnings
Sh

In [58]:
print("Real Headline : ", summary[1],"\n Predicted Summary : ", summarize(article[1]))


Real Headline :  <SOS> The Northern Ireland economy should grow at 2.4% in 2014 and 2015 but 2,400 public sector jobs could be lost in that period, Danske Bank has forecast. <EOS> 
 Predicted Summary :  the number of people killed last year fell on the lowest quarter of the year of the year ending december according to estimates
